In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.genmod.families import family
from statsmodels.genmod.families import links
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import random
from IPython.display import display

# Set random seed for reproducibility
np.random.seed(123)
random.seed(123)

# Function to generate data for target trial emulation
def generate_synthetic_data(n_patients=10000):
    """
    Generate synthetic data for target trial emulation.
    
    Parameters:
    -----------
    n_patients : int
        Number of patients to simulate
        
    Returns:
    --------
    DataFrame containing simulated patient data
    """
    # Generate baseline covariates
    age = np.random.normal(65, 10, n_patients)  # Age centered around 65
    male = np.random.binomial(1, 0.5, n_patients)  # Gender
    bmi = np.random.normal(27, 5, n_patients)  # BMI
    diabetes = np.random.binomial(1, 0.3, n_patients)  # Diabetes status
    hypertension = np.random.binomial(1, 0.4, n_patients)  # Hypertension status
    smoking = np.random.binomial(1, 0.25, n_patients)  # Smoking status
    
    # Generate treatment assignment based on covariates
    # Higher probability of treatment for older, male patients with comorbidities
    logit_treat = -2 + 0.02*age + 0.5*male + 0.03*bmi + 0.7*diabetes + 0.6*hypertension + 0.4*smoking
    p_treat = 1 / (1 + np.exp(-logit_treat))
    treatment = np.random.binomial(1, p_treat, n_patients)
    
    # Generate outcome based on treatment and covariates
    # Treatment has a protective effect (negative coefficient)
    logit_outcome = -3 + 0.03*age + 0.3*male + 0.02*bmi + 0.6*diabetes + 0.5*hypertension + 0.3*smoking - 0.7*treatment
    p_outcome = 1 / (1 + np.exp(-logit_outcome))
    outcome = np.random.binomial(1, p_outcome, n_patients)
    
    # Generate time-to-event data
    # Higher hazard for older patients with comorbidities, lower hazard for treated
    lambda_event = np.exp(-5 + 0.02*age + 0.3*male + 0.01*bmi + 0.5*diabetes + 0.4*hypertension + 0.2*smoking - 0.6*treatment)
    time_to_event = np.random.exponential(1/lambda_event)
    
    # Generate censoring times
    censoring_time = np.random.exponential(1/0.1, n_patients)
    
    # Take minimum of event time and censoring time
    observed_time = np.minimum(time_to_event, censoring_time)
    event = (time_to_event <= censoring_time).astype(int)
    
    # Create dataframe
    data = pd.DataFrame({
        'patient_id': range(1, n_patients + 1),
        'age': age,
        'male': male,
        'bmi': bmi,
        'diabetes': diabetes,
        'hypertension': hypertension,
        'smoking': smoking,
        'treatment': treatment,
        'outcome': outcome,
        'time': observed_time,
        'event': event
    })
    
    return data

# Generate the data
data = generate_synthetic_data(10000)

# Display the first few rows of the data
print("Sample of synthetic patient data:")
display(data.head())

# Descriptive statistics by treatment group
def descriptive_stats_by_treatment(data):
    """Generate descriptive statistics by treatment group"""
    stats_dict = {}
    
    for var in ['age', 'male', 'bmi', 'diabetes', 'hypertension', 'smoking', 'outcome', 'time', 'event']:
        # Calculate statistics for treated group
        treated_stats = data[data.treatment == 1][var].describe()
        
        # Calculate statistics for untreated group
        untreated_stats = data[data.treatment == 0][var].describe()
        
        # For binary variables, calculate percentages
        if var in ['male', 'diabetes', 'hypertension', 'smoking', 'outcome', 'event']:
            treated_pct = data[data.treatment == 1][var].mean() * 100
            untreated_pct = data[data.treatment == 0][var].mean() * 100
            
            # Calculate p-value using chi-square test
            contingency_table = pd.crosstab(data['treatment'], data[var])
            chi2, p_value = stats.chi2_contingency(contingency_table)[:2]
            
            stats_dict[var] = {
                'Treated': f"{treated_pct:.1f}%",
                'Untreated': f"{untreated_pct:.1f}%",
                'p-value': f"{p_value:.4f}"
            }
        else:
            # Calculate p-value using t-test
            t_stat, p_value = stats.ttest_ind(
                data[data.treatment == 1][var].dropna(),
                data[data.treatment == 0][var].dropna()
            )
            
            stats_dict[var] = {
                'Treated': f"{treated_stats['mean']:.2f} ± {treated_stats['std']:.2f}",
                'Untreated': f"{untreated_stats['mean']:.2f} ± {untreated_stats['std']:.2f}",
                'p-value': f"{p_value:.4f}"
            }
            
    return pd.DataFrame(stats_dict).T

# Calculate descriptive statistics
desc_stats = descriptive_stats_by_treatment(data)
print("\nDescriptive statistics by treatment group:")
display(desc_stats)

# Calculate propensity scores
def calculate_propensity_scores(data, covariates):
    """
    Calculate propensity scores using logistic regression
    
    Parameters:
    -----------
    data : DataFrame
        Patient data
    covariates : list
        List of covariate column names
        
    Returns:
    --------
    Series containing propensity scores
    """
    X = data[covariates]
    y = data['treatment']
    
    # Fit logistic regression model
    logistic_model = LogisticRegression(max_iter=1000)
    logistic_model.fit(X, y)
    
    # Calculate propensity scores
    propensity_scores = logistic_model.predict_proba(X)[:, 1]
    
    return propensity_scores

# Calculate propensity scores
covariates = ['age', 'male', 'bmi', 'diabetes', 'hypertension', 'smoking']
data['propensity_score'] = calculate_propensity_scores(data, covariates)

# Visualize propensity score distributions
plt.figure(figsize=(10, 6))
sns.histplot(data=data, x='propensity_score', hue='treatment', bins=30, element='step', common_norm=False)
plt.title('Propensity Score Distribution by Treatment Group')
plt.xlabel('Propensity Score')
plt.ylabel('Count')
plt.show()

# Perform inverse probability weighting (IPW)
def inverse_probability_weighting(data):
    """
    Perform inverse probability of treatment weighting (IPTW)
    
    Parameters:
    -----------
    data : DataFrame
        Patient data with propensity scores
        
    Returns:
    --------
    DataFrame with added IPW weights
    """
    # Create a copy of the data
    weighted_data = data.copy()
    
    # Calculate weights
    weighted_data['ipw'] = np.where(
        weighted_data['treatment'] == 1,
        1 / weighted_data['propensity_score'],
        1 / (1 - weighted_data['propensity_score'])
    )
    
    # Stabilize weights
    treated_prob = weighted_data['treatment'].mean()
    weighted_data['ipw_stabilized'] = np.where(
        weighted_data['treatment'] == 1,
        treated_prob / weighted_data['propensity_score'],
        (1 - treated_prob) / (1 - weighted_data['propensity_score'])
    )
    
    return weighted_data

# Apply IPW
weighted_data = inverse_probability_weighting(data)

# Check balance after weighting
def check_covariate_balance(data, weighted_data, covariates):
    """
    Check covariate balance before and after weighting
    
    Parameters:
    -----------
    data : DataFrame
        Original patient data
    weighted_data : DataFrame
        Weighted patient data
    covariates : list
        List of covariate column names
        
    Returns:
    --------
    DataFrame with standardized mean differences
    """
    balance_results = []
    
    for var in covariates:
        # Unweighted standardized difference
        treated_mean = data[data.treatment == 1][var].mean()
        treated_var = data[data.treatment == 1][var].var()
        
        untreated_mean = data[data.treatment == 0][var].mean()
        untreated_var = data[data.treatment == 0][var].var()
        
        pooled_sd = np.sqrt((treated_var + untreated_var) / 2)
        unweighted_smd = abs(treated_mean - untreated_mean) / pooled_sd if pooled_sd > 0 else 0
        
        # Weighted standardized difference
        treated_weight_sum = weighted_data[weighted_data.treatment == 1]['ipw_stabilized'].sum()
        untreated_weight_sum = weighted_data[weighted_data.treatment == 0]['ipw_stabilized'].sum()
        
        weighted_treated_mean = (weighted_data[weighted_data.treatment == 1][var] * 
                                weighted_data[weighted_data.treatment == 1]['ipw_stabilized']).sum() / treated_weight_sum
        
        weighted_untreated_mean = (weighted_data[weighted_data.treatment == 0][var] * 
                                  weighted_data[weighted_data.treatment == 0]['ipw_stabilized']).sum() / untreated_weight_sum
        
        weighted_smd = abs(weighted_treated_mean - weighted_untreated_mean) / pooled_sd if pooled_sd > 0 else 0
        
        balance_results.append({
            'Variable': var,
            'Unweighted_SMD': unweighted_smd,
            'Weighted_SMD': weighted_smd
        })
    
    return pd.DataFrame(balance_results)

# Check balance
balance_df = check_covariate_balance(data, weighted_data, covariates)
print("\nCovariate balance before and after weighting:")
display(balance_df)

# Plot balance
plt.figure(figsize=(10, 6))
balance_plot = pd.melt(balance_df, id_vars=['Variable'], 
                       value_vars=['Unweighted_SMD', 'Weighted_SMD'],
                       var_name='Method', value_name='Standardized Mean Difference')

sns.barplot(data=balance_plot, x='Variable', y='Standardized Mean Difference', hue='Method')
plt.axhline(y=0.1, color='r', linestyle='--', label='Threshold (0.1)')
plt.title('Covariate Balance Before and After Weighting')
plt.legend(title='')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Estimate treatment effect using weighted regression
def estimate_treatment_effect(weighted_data, covariates):
    """
    Estimate treatment effect using weighted regression
    
    Parameters:
    -----------
    weighted_data : DataFrame
        Weighted patient data
    covariates : list
        List of covariate column names
        
    Returns:
    --------
    Fitted model and summary
    """
    # Prepare data for regression
    X = weighted_data[['treatment'] + covariates].copy()
    X = sm.add_constant(X)
    y = weighted_data['outcome']
    weights = weighted_data['ipw_stabilized']
    
    # Fit weighted logistic regression
    model = sm.GLM(y, X, family=family.Binomial(link=links.logit()), freq_weights=weights)
    result = model.fit()
    
    # Convert log odds to odds ratios
    params = result.params.copy()
    conf_int = result.conf_int()
    
    odds_ratios = np.exp(params)
    odds_ratios_ci = np.exp(conf_int)
    
    # Create summary DataFrame
    summary_df = pd.DataFrame({
        'Odds Ratio': odds_ratios,
        'Lower 95% CI': odds_ratios_ci[0],
        'Upper 95% CI': odds_ratios_ci[1],
        'p-value': result.pvalues
    })
    
    return result, summary_df

# Estimate treatment effect
model, effect_summary = estimate_treatment_effect(weighted_data, covariates)
print("\nTreatment effect estimation:")
display(effect_summary)

# Time-to-event analysis
def time_to_event_analysis(weighted_data):
    """
    Perform time-to-event analysis using weighted Cox regression
    
    Parameters:
    -----------
    weighted_data : DataFrame
        Weighted patient data
        
    Returns:
    --------
    Estimated hazard ratio for treatment effect
    """
    # Note: For a full implementation, you would use lifelines or other survival analysis package
    # Since we're just providing a conceptual Python equivalent, we'll simplify this part
    
    # The following is a placeholder to represent the general approach
    print("\nTime-to-event analysis:")
    print("Note: In a complete implementation, you would use the lifelines package")
    print("to perform a weighted Cox proportional hazards regression.")
    print("\nExample code:")
    print("from lifelines import CoxPHFitter")
    print("cox_model = CoxPHFitter()")
    print("cox_model.fit(weighted_data, duration_col='time', event_col='event', weights_col='ipw_stabilized')")
    print("cox_model.print_summary()")
    
    # For simplicity, report the ratio of event rates as crude approximation
    treated_events = weighted_data[weighted_data.treatment == 1]['event'].sum()
    treated_time = weighted_data[weighted_data.treatment == 1]['time'].sum()
    treated_rate = treated_events / treated_time
    
    untreated_events = weighted_data[weighted_data.treatment == 0]['event'].sum()
    untreated_time = weighted_data[weighted_data.treatment == 0]['time'].sum()
    untreated_rate = untreated_events / untreated_time
    
    crude_hr = treated_rate / untreated_rate
    
    return {
        'Crude Hazard Ratio': crude_hr,
        'Treated Event Rate': treated_rate,
        'Untreated Event Rate': untreated_rate
    }

# Perform time-to-event analysis
survival_results = time_to_event_analysis(weighted_data)
print("\nApproximate time-to-event results:")
for key, value in survival_results.items():
    print(f"{key}: {value:.4f}")

# Sensitivity analysis
def sensitivity_analysis(data, covariates, trim_threshold=0.05):
    """
    Perform sensitivity analysis by trimming extreme propensity scores
    
    Parameters:
    -----------
    data : DataFrame
        Patient data
    covariates : list
        List of covariate column names
    trim_threshold : float
        Threshold for trimming propensity scores
        
    Returns:
    --------
    DataFrame with trimmed data and weights
    """
    # Create trimmed dataset
    trimmed_data = data[(data['propensity_score'] >= trim_threshold) & 
                        (data['propensity_score'] <= (1 - trim_threshold))].copy()
    
    print(f"\nTrimmed {len(data) - len(trimmed_data)} patients with extreme propensity scores")
    print(f"Remaining sample size: {len(trimmed_data)}")
    
    # Recalculate weights
    trimmed_data['ipw'] = np.where(
        trimmed_data['treatment'] == 1,
        1 / trimmed_data['propensity_score'],
        1 / (1 - trimmed_data['propensity_score'])
    )
    
    treated_prob = trimmed_data['treatment'].mean()
    trimmed_data['ipw_stabilized'] = np.where(
        trimmed_data['treatment'] == 1,
        treated_prob / trimmed_data['propensity_score'],
        (1 - treated_prob) / (1 - trimmed_data['propensity_score'])
    )
    
    return trimmed_data

# Perform sensitivity analysis
trimmed_data = sensitivity_analysis(data, covariates, trim_threshold=0.05)

# Re-estimate treatment effect with trimmed data
trimmed_model, trimmed_effect = estimate_treatment_effect(trimmed_data, covariates)
print("\nTreatment effect estimation after trimming:")
display(trimmed_effect)

# Subgroup analysis
def subgroup_analysis(weighted_data, subgroup_var):
    """
    Perform subgroup analysis for treatment effect
    
    Parameters:
    -----------
    weighted_data : DataFrame
        Weighted patient data
    subgroup_var : str
        Variable name to define subgroups
        
    Returns:
    --------
    Dictionary with subgroup treatment effects
    """
    subgroup_results = {}
    
    for subgroup_value in weighted_data[subgroup_var].unique():
        subgroup_data = weighted_data[weighted_data[subgroup_var] == subgroup_value]
        
        # Simple calculation of risk difference
        treated = subgroup_data[subgroup_data.treatment == 1]
        untreated = subgroup_data[subgroup_data.treatment == 0]
        
        treated_risk = (treated['outcome'] * treated['ipw_stabilized']).sum() / treated['ipw_stabilized'].sum()
        untreated_risk = (untreated['outcome'] * untreated['ipw_stabilized']).sum() / untreated['ipw_stabilized'].sum()
        
        risk_difference = treated_risk - untreated_risk
        
        subgroup_results[f"{subgroup_var}={subgroup_value}"] = {
            'Treated Risk': treated_risk,
            'Untreated Risk': untreated_risk,
            'Risk Difference': risk_difference,
            'Sample Size': len(subgroup_data)
        }
    
    return subgroup_results

# Perform subgroup analysis for diabetes
diabetes_subgroups = subgroup_analysis(weighted_data, 'diabetes')
print("\nSubgroup analysis by diabetes status:")
for subgroup, results in diabetes_subgroups.items():
    print(f"\n{subgroup}:")
    for key, value in results.items():
        print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

# Perform matching as alternative to weighting
def perform_matching(data, covariates, caliper=0.2):
    """
    Perform propensity score matching
    
    Parameters:
    -----------
    data : DataFrame
        Patient data
    covariates : list
        List of covariate column names
    caliper : float
        Caliper width for matching
        
    Returns:
    --------
    DataFrame with matched pairs
    """
    # Create a copy of the data
    df = data.copy()
    
    # Initialize empty dataframe for matched pairs
    matched_pairs = pd.DataFrame()
    
    # Get treated and untreated patients
    treated = df[df.treatment == 1].copy().reset_index(drop=True)
    untreated = df[df.treatment == 0].copy().reset_index(drop=True)
    
    # Calculate propensity score SD for caliper
    ps_sd = df['propensity_score'].std()
    caliper_width = caliper * ps_sd
    
    # Create arrays to keep track of matched status
    treated_matched = np.zeros(len(treated), dtype=bool)
    untreated_matched = np.zeros(len(untreated), dtype=bool)
    
    # For each treated patient, find the best match
    for i in range(len(treated)):
        if treated_matched[i]:
            continue
            
        treated_ps = treated.loc[i, 'propensity_score']
        
        # Calculate absolute differences in propensity scores
        diffs = np.abs(untreated['propensity_score'] - treated_ps)
        
        # Find eligible matches (within caliper and not already matched)
        eligible = (~untreated_matched) & (diffs <= caliper_width)
        
        if np.any(eligible):
            # Find the best match
            best_match_idx = diffs[eligible].idxmin()
            
            # Mark as matched
            treated_matched[i] = True
            untreated_matched[best_match_idx] = True
            
            # Add pair to result
            pair = pd.concat([treated.loc[[i]], untreated.loc[[best_match_idx]]])
            matched_pairs = pd.concat([matched_pairs, pair])
    
    print(f"\nMatching results:")
    print(f"Treated patients matched: {treated_matched.sum()} out of {len(treated)}")
    print(f"Matching rate: {treated_matched.sum()/len(treated):.2%}")
    
    return matched_pairs

# Perform matching
matched_data = perform_matching(data, covariates)

# Analyze matched data
def analyze_matched_data(matched_data):
    """
    Analyze treatment effect in matched data
    
    Parameters:
    -----------
    matched_data : DataFrame
        Matched patient data
        
    Returns:
    --------
    Treatment effect estimate
    """
    # Simple difference in outcome rates
    treated = matched_data[matched_data.treatment == 1]
    untreated = matched_data[matched_data.treatment == 0]
    
    treated_outcome_rate = treated['outcome'].mean()
    untreated_outcome_rate = untreated['outcome'].mean()
    
    risk_difference = treated_outcome_rate - untreated_outcome_rate
    odds_ratio = (treated_outcome_rate / (1 - treated_outcome_rate)) / (untreated_outcome_rate / (1 - untreated_outcome_rate))
    
    return {
        'Treated Outcome Rate': treated_outcome_rate,
        'Untreated Outcome Rate': untreated_outcome_rate,
        'Risk Difference': risk_difference,
        'Odds Ratio': odds_ratio
    }

# Analyze matched data if we have any matches
if len(matched_data) > 0:
    matched_results = analyze_matched_data(matched_data)
    print("\nTreatment effect in matched data:")
    for key, value in matched_results.items():
        print(f"{key}: {value:.4f}")

# Create a function to summarize results
def summarize_results(data, weighted_data, effect_summary, subgroup_results, survival_results):
    """Summarize all results of the target trial emulation"""
    print("\n" + "="*50)
    print("TARGET TRIAL EMULATION SUMMARY")
    print("="*50)
    
    # Dataset information
    print(f"\nDataset: {len(data)} patients")
    print(f"Treatment group: {data['treatment'].sum()} patients ({data['treatment'].mean()*100:.1f}%)")
    print(f"Control group: {len(data) - data['treatment'].sum()} patients ({(1-data['treatment'].mean())*100:.1f}%)")
    
    # Primary outcome
    print(f"\nPrimary outcome event rate: {data['outcome'].mean()*100:.1f}%")
    
    # Main treatment effect
    treatment_or = effect_summary.loc['treatment', 'Odds Ratio']
    treatment_ci_lower = effect_summary.loc['treatment', 'Lower 95% CI']
    treatment_ci_upper = effect_summary.loc['treatment', 'Upper 95% CI']
    treatment_p = effect_summary.loc['treatment', 'p-value']
    
    print(f"\nPrimary analysis - Treatment effect:")
    print(f"Odds Ratio: {treatment_or:.2f} (95% CI: {treatment_ci_lower:.2f} - {treatment_ci_upper:.2f}), p = {treatment_p:.4f}")
    
    # Interpretation based on direction of effect
    if treatment_or < 1 and treatment_p < 0.05:
        print("Interpretation: Treatment significantly reduces the odds of the outcome")
    elif treatment_or > 1 and treatment_p < 0.05:
        print("Interpretation: Treatment significantly increases the odds of the outcome")
    else:
        print("Interpretation: No statistically significant effect of treatment detected")
    
    # Survival analysis
    if survival_results:
        print(f"\nTime-to-event analysis - Crude Hazard Ratio: {survival_results['Crude Hazard Ratio']:.2f}")
    
    # Subgroup analysis
    if subgroup_results:
        print("\nSubgroup analysis:")
        for subgroup, results in subgroup_results.items():
            print(f"  {subgroup}: Risk Difference = {results['Risk Difference']:.4f}")

# Summarize all results
summarize_results(data, weighted_data, effect_summary, diabetes_subgroups, survival_results)

ModuleNotFoundError: No module named 'statsmodels'